In [1]:
%config IPCompleter.use_jedi = False

# Libraries

In [36]:
import pandas as pd
import numpy as np
import transliterate
import googletrans
import geonamescache
import geopy
from geopy.geocoders import Nominatim

In [3]:
pd.options.display.max_rows = 999

# Constants

In [4]:
USSR_codes = ['RU', 'LV', 'AM', 'UA', 'BY', 'RU-DA']

In [5]:
airport_dict = {
    'Tolmachevo': 'Novosibirsk', 
    'Koltsovo': 'Ekaterinburg',
    'Kurumoch': 'Samara',
    'Vityazevo': 'Anapa',
    'Platov': 'Rostov-na-Donu',
    'Vnukovo': 'Moscow',
    'Domodedovo': 'Moscow',
    "Sheremet'evo": 'Moscow',
    "Moskva Oktjabr'skaja": 'Moscow',
    'Nizhnevo': 'Barnaul' #Because the flight number matches Barnaul flight
}

# Functions

In [6]:
def find_сity_name(phrase):
    if any([x.lower() in phrase.lower() 
        for x in np.unique(sng_cities.city)]):
        indeces = np.where([x.lower() in phrase.lower() 
        for x in sng_cities.city]
                )
        corrected_names = np.unique(sng_cities.iloc[indeces].city)
        corrected_names = sorted(corrected_names, key = lambda x: len(x), reverse=True)
        return(corrected_names[0])
    else:
        indeces = np.where([x.lower() in phrase.lower() 
            for x in sng_cities.alternative_name]
                    )
        corrected_names = np.unique(sng_cities.iloc[indeces].city)
        corrected_names = sorted(corrected_names, key = lambda x: len(x), reverse=True)
        if len(corrected_names) == 0:
            return np.NaN
        else: 
            return(corrected_names[0])

In [7]:
def find_сity_name(phrase, check_main_names = True):
    if check_main_names:
        city = string_against_set(phrase, sng_cities.city.values)
    else:
        city = None
    if city is None:
        city = string_against_set(phrase, sng_cities.alternative_name.values)
        if city is not None:
            city = np.unique(sng_cities.query('alternative_name == @city').city.values)[0]
    return(city)

In [8]:
def string_against_set(string, set_of_strings):
    indeces = np.where([x.lower() in string.lower() 
                        for x in set_of_strings])[0]
    if len(indeces) == 0:
        return None
    else:
        names = np.unique(set_of_strings[indeces])
        names = sorted(names, key = lambda x: len(x), reverse=True)
        return(names[0])

In [9]:
def airport_to_city(string):
    city = string_against_set(string, np.array(list(airport_dict.keys()))
                             )
    if city is not None:
        return(airport_dict[city])
    else:
        return(string)

# Read data

In [10]:
file = pd.ExcelFile("../data/_FSB Poison Squad Travel History_no_comments.ods", engine="odf")
names = file.sheet_names  # see all sheet names

In [11]:
all_data = pd.DataFrame()
for name in names[1:]:
    data = file.parse(
            sheet_name=name,
            keep_default_na = True, 
            na_filter = True,
            encoding='cp1251'
                )
    data.Date = pd.to_datetime(data.Date)
    data.dropna(how = 'all', inplace = True)
    data['Name'] = name
    all_data = pd.concat((
        all_data,
        data
    ),
        ignore_index = True
    )

In [12]:
all_data['Departure_original'] = all_data['Departure']

In [13]:
all_data['Arrival_original'] = all_data['Arrival']

# Harmonize

In [14]:
all_data.Departure = all_data.Departure.apply(lambda x: x.split(',')[0])
all_data.Arrival = all_data.Arrival.apply(lambda x: x.split(',')[0])

## Transliterate to english

In [15]:
translate_to_en = lambda x: transliterate.translit(x, "ru", reversed=True).lower()
translate_to_ru = lambda x: transliterate.translit(x, "ru", reversed=False).lower()
get_language = lambda x: 'ru' if transliterate.detect_language(x) == 'ru' else 'en'

In [16]:
all_data['language'] = all_data.Departure.apply(get_language)

In [17]:
all_data['Departure_en'] = all_data.Departure
all_data['Arrival_en'] = all_data.Arrival

all_data.loc[all_data.language == 'ru', 'Departure_en'] = all_data.query('language == "ru"').Departure.apply(translate_to_en)
all_data.loc[all_data.language == 'ru', 'Arrival_en'] = all_data.query('language == "ru"').Arrival.apply(translate_to_en)

## Replace airport names with city names

In [18]:
all_data.Departure_en = all_data.Departure_en.apply(airport_to_city)
all_data.Arrival_en = all_data.Arrival_en.apply(airport_to_city)

## Transliterate to Russian

In [19]:
all_data['Departure_ru'] = all_data.Departure
all_data['Arrival_ru'] = all_data.Arrival

all_data.loc[all_data.language == 'en', 'Departure_ru'] = all_data.query('language == "en"').Departure.apply(translate_to_ru)
all_data.loc[all_data.language == 'en', 'Arrival_ru'] = all_data.query('language == "en"').Arrival.apply(translate_to_ru)

## Extract city names from db

In [20]:
gc = geonamescache.GeonamesCache()

cities = gc.get_cities()

cities_df = pd.DataFrame(
[[x, cities[x]['name'], cities[x]['countrycode']] for x in cities.keys()],
columns = ('code', 'city', 'country')
)

sng_cities = cities_df.query('country in @USSR_codes')
sng_cities = pd.DataFrame(
[[x, 
 cities[x]['name'], 
 cities[x]['countrycode'],
 cities[x]['alternatenames'] + [cities[x]['name']] + [cities[x]['name'].replace('-', ' ')]] for x in sng_cities.code],
columns = ('code', 'city', 'country', 'alternative_name')
)

In [21]:
sng_cities = sng_cities.explode('alternative_name').reset_index(drop=True)

sng_cities.loc[
    sng_cities.alternative_name == '', 
    'alternative_name'] = sng_cities.loc[
                                sng_cities.alternative_name == '', 
                                'city']

Disambiguate

In [22]:
sng_cities.drop(index = sng_cities[(sng_cities.city == "Rostov-na-Donu") &
                                   (sng_cities.alternative_name == 'Rostov')].index,
               inplace = True)

In [23]:
sng_cities.drop(index = sng_cities[(sng_cities.city == "Korolev") &
                                   (sng_cities.alternative_name == 'Kaliningrad')].index,
               inplace = True)

## Add city names to data

First try to find english names

In [24]:
all_data.loc[all_data.language == 'en',
         'Departure_corrected'] = all_data.loc[all_data.language == 'en',
                                          'Departure_en'].apply(find_сity_name)

all_data.loc[all_data.language == 'en',
         'Arrival_corrected'] = all_data.loc[all_data.language == 'en',
                                         'Arrival_en'].apply(find_сity_name)

In [25]:
all_data.loc[all_data.language == 'ru',
         'Departure_corrected'] = all_data.loc[all_data.language == 'ru',
                                          'Departure_en'].apply(lambda x: find_сity_name(x, 
                                                                                         check_main_names=False))

all_data.loc[all_data.language == 'ru',
         'Arrival_corrected'] = all_data.loc[all_data.language == 'ru',
                                         'Arrival_en'].apply(lambda x: find_сity_name(x, 
                                                                                         check_main_names=False))

Now try russian names

In [26]:
all_data.loc[
    pd.isna(all_data.Departure_corrected),
    'Departure_corrected'] = all_data[
                                pd.isna(all_data.Departure_corrected)
                            ].Departure_ru.apply(lambda x: find_сity_name(x, check_main_names=False))

In [27]:
all_data.loc[
    pd.isna(all_data.Arrival_corrected),
    'Arrival_corrected'] = all_data[
                                pd.isna(all_data.Arrival_corrected)
                            ].Arrival_ru.apply(lambda x: find_сity_name(x, check_main_names=False))

In [28]:
all_data.loc[pd.isna(all_data.Arrival_corrected), 
             'Arrival_corrected'] = all_data.loc[pd.isna(all_data.Arrival_corrected),
                                                 'Arrival_en']

In [29]:
all_data.loc[pd.isna(all_data.Departure_corrected),
        'Departure_corrected'] = all_data.loc[pd.isna(all_data.Departure_corrected),
                                            'Departure_en']

## Manual correction

In [30]:
for name in ['Osnovnoy', 'zapadnaja dvina', 'pashkovskaja']:
    for col_name in ['Arrival', 'Departure']:
        all_data.loc[all_data[f'{col_name}_en'] == name,
                     f'{col_name}_corrected'] = all_data.loc[all_data[f'{col_name}_en'] == name,
                                                             f'{col_name}_en']

In [31]:
for col_name in ['Arrival', 'Departure']:
        all_data.loc[all_data[f'{col_name}_en'] == "brjansk orlovskij",
                     f'{col_name}_corrected'] = 'Bryansk'

Capitalize

In [32]:
all_data.Departure_corrected = all_data.Departure_corrected.apply(lambda x: x.title())
all_data.Arrival_corrected = all_data.Arrival_corrected.apply(lambda x: x.title())

Add country

In [33]:
all_data['Departure_country'] = all_data.Departure_corrected.apply(lambda x: np.unique(sng_cities.query('city == @x').country)
                                  )

all_data.Departure_country = all_data.Departure_country.explode()

In [34]:
all_data['Arrival_country'] = all_data.Arrival_corrected.apply(lambda x: np.unique(sng_cities.query('city == @x').country)
                                  )

all_data.Arrival_country = all_data.Arrival_country.explode()

In [73]:
all_data_to_save = all_data[['Date', 'Name', 'Departure_original', 'Arrival_original', 'Notes', 
                             'Departure_corrected', 'Arrival_corrected', 'Departure_country',
                            'Arrival_country']]

In [77]:
all_data_to_save.to_csv('../data/FSB_poison_squad_travel_history_harmonized.csv', index=False)

## Add coordinates

In [37]:
geolocator = Nominatim(user_agent="my_user_agent")

In [62]:
all_data.Arrival_corrected = all_data.Arrival_corrected.astype(str)
all_data.Departure_corrected = all_data.Departure_corrected.astype(str)

In [67]:
all_data['Arrival_location'] = all_data.Arrival_corrected + ','+ all_data.Arrival_country

In [ ]:
all_data['Arrival_loc'] = all_data['Arrival_location'].apply(
    geolocator.geocode
)